In [1]:
import numpy as np
import pandas as pd
import warnings
import gc
import tensorflow as tf
from tensorflow import keras

import random
from random import choice
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, GRU, Concatenate, Embedding, Flatten, Activation, Dropout
from sklearn.model_selection import KFold
from tensorflow.python.client import device_lib
warnings.filterwarnings('ignore')
import random

In [2]:
import gc

## Data Preprocessing

In [3]:
# load the data
import pandas as pd
dataframe = pd.read_csv('Practice_Log_Demographics.csv', low_memory=False)
dataframe.head()

,Unnamed: 0,chapter_label,sub_chapter_label,question_name,user_id.x,term,STDNT_SEX_CD,NonNativeEnglish,White,Asian,...,classGraded,classHonors,Pass_Fail,parentsGraduateEdu,minorityGroup,q,day's_available_flashcards,start_practice,end_practice,days_offset
0,1,Sort,intro-SortingwithSortandSorted,ee_sort_04,148,WN 2018,1,0,1,0,...,1,0,0,0,0,0,18,2018/3/4 20:00,2018/3/4 20:00,45
1,2,Functions,Returningavaluefromafunction,test_questionfunctions_2_3,148,WN 2018,1,0,1,0,...,1,0,0,0,0,4,16,2018/2/10 11:57,2018/2/10 11:58,23
2,3,RESTAPIs,functionswithrestapis,test_question_functions_rest_apis_1,148,WN 2018,1,0,1,0,...,1,0,0,0,0,1,2,2018/3/12 17:03,2018/3/12 17:04,53
3,4,SimplePythonData,FunctionCalls,exercise_functionCalls_1,148,WN 2018,1,0,1,0,...,1,0,0,0,0,0,10,2018/1/30 14:15,2018/1/30 14:15,12
4,5,DictionaryAccumulation,AccumulatingaMaximumValue,ps_04_07,148,WN 2018,1,0,1,0,...,1,0,0,0,0,0,23,2018/3/3 14:04,2018/3/3 14:04,44


In [4]:
#sort data based on timestamp
dataframe = dataframe.sort_values(by=['start_practice'])

In [5]:
FEATURES = ['chapter_label', 'sub_chapter_label','question_name','user_id','term',
            'STDNT_SEX_CD', 
            'NonNativeEnglish',
            'White',
            'Asian',
            'WhiteOrAsian',
            'Hispanic',
            'AfricanAmerican',
            'OtherEthnicities',
            'NonWhiteOrAsian',
            'STDNT_CTZN_STAT_CD', 'international', 
            'gradingType',
            'birthYear',
            'exclClassCumGPA',
            'Freshman',
            'Junior',
            'Sophomore',
            'Senior',
            'termCreditsGPA',
            'termCreditsNoGPA',
            'athlete_1',
            'honorsPro',
            'LSA', 'programBusiness', 'programEngineering', 
            'programInformation', 'programOther',
            'HSCalculusTaken', 
            'highSchoolGPA', 
            'majorsCount', 'minorsCount',
            'PREV_TERM_CUM_GPA',
            'classGraded', 'classHonors', 
            'Pass_Fail', 
            'parentsGraduateEdu',  'minorityGroup', 
            'q',
            'available_flashcards', 
            'start_practice', 
            'end_practice',
            'days_offset']

In [6]:
dataframe['available_flashcards'] = dataframe["day's_available_flashcards"][:]
dataframe = dataframe.drop(["day's_available_flashcards"], axis=1)
dataframe['user_id'] = dataframe["user_id.x"][:]
dataframe = dataframe.drop(["user_id.x"], axis=1)

In [7]:
gc.collect()

102

In [8]:
dataframe = dataframe[FEATURES]
gc.collect()

20

In [9]:
dataframe = dataframe.fillna(0)

In [10]:
#label encode term, chapter_label, question_name, sub_chapter_label columns
dataframe['term'] = dataframe['term'].astype('category')
dataframe['user_id'] = dataframe['user_id'].astype(int)
dataframe['user_id'] = dataframe['user_id'].astype(str)
dataframe['user_id'] = dataframe['term'].str.cat(dataframe['user_id'], sep=':')
dataframe['user_id'] = dataframe['user_id'].astype('category')
dataframe['chapter_label'] = dataframe['chapter_label'].astype('category')
dataframe['sub_chapter_label'] = dataframe['sub_chapter_label'].astype('category')
dataframe['question_name'] = dataframe['question_name'].astype('category')


In [11]:
#calculate time_lag and prev_time_elapsed
dataframe['prev_time_elapsed'] = None
dataframe['time_lag'] = None
dataframe['time_lag'] = dataframe['time_lag'].astype(np.float)
dataframe['prev_time_elapsed'] = dataframe['prev_time_elapsed'].astype(np.float)
dataframe.start_practice = pd.to_datetime(dataframe.start_practice, format='%Y-%m-%d %H:%M:%S')
dataframe.end_practice = pd.to_datetime(dataframe.end_practice, format='%Y-%m-%d %H:%M:%S')
dataframe['dif'] = dataframe.end_practice - dataframe.start_practice
dataframe['dif'] = dataframe['dif'] /np.timedelta64(1, 's')
dataframe['answer_correct'] = np.where((dataframe['q']==5) & (dataframe['dif'] <= 60), 1, 0)

In [13]:
#drop column end_practice
dataframe.drop(columns=['end_practice'], inplace=True)

In [14]:
# calculate the age feature
dataframe['term_value'] = [int(ele[3:]) for ele in dataframe['term']]
dataframe['age'] = dataframe['term_value'] - dataframe['birthYear']

In [15]:
# drop term_value and birthYear column
dataframe.drop(columns=['term_value', 'birthYear'], inplace=True)

In [16]:
# convert minors_count to int value
new_minors_count = []
for i in dataframe['minorsCount']:
  if i == 0 or i == '0':
    new_minors_count.append(0)
  elif i == '1 Minor':
    new_minors_count.append(1)
  else:
    new_minors_count.append(2)

dataframe['minorsCount'] = new_minors_count

In [17]:
print("we have ", dataframe['user_id'].nunique()," users in total.")

we have  1110  users in total.


In [18]:
for category in ['term','chapter_label', 'sub_chapter_label', 'question_name']:
  dataframe[category] =  dataframe[category].cat.codes

In [19]:
NUMERIC_FEATURE =  ['age',
            'exclClassCumGPA',
            'termCreditsGPA',
            'termCreditsNoGPA',
            'highSchoolGPA', 
            'majorsCount', 'minorsCount',
            'PREV_TERM_CUM_GPA',
            'available_flashcards', 
            'days_offset', 
            'prev_time_elapsed',
             'time_lag']
# z-score normalize the numerical features
for f in NUMERIC_FEATURE:
  m = dataframe[f].mean()
  std = dataframe[f].std()
  dataframe[f] = (dataframe[f] - m)/std

In [20]:
FEATURE_TRANS =  ['answer_correct', 'chapter_label', 'sub_chapter_label','question_name','user_id','term',
                  'STDNT_SEX_CD', 
                    'White','Asian','NonWhiteOrAsian',
            'STDNT_CTZN_STAT_CD', 'international', 
            'age',
            'exclClassCumGPA',
           'Freshman',
            'Junior',
            'Sophomore',
            'Senior',
            'termCreditsGPA',
            'termCreditsNoGPA',
            'athlete_1',
            'honorsPro',
            'LSA', 'programBusiness', 'programEngineering', 
            'programInformation', 'programOther',
            'HSCalculusTaken', 
            'highSchoolGPA', 
            'majorsCount', 'minorsCount',
            'PREV_TERM_CUM_GPA',
            'classGraded', 'classHonors', 
            'Pass_Fail', 
            'parentsGraduateEdu',  'minorityGroup', 
            'available_flashcards', 
            'days_offset', 'prev_time_elapsed',
             'time_lag']
grouped_data = dataframe[FEATURE_TRANS].groupby(['user_id']).apply(lambda r: (
                r['answer_correct'],
                r['term'],
                r['chapter_label'],
                r['sub_chapter_label'],
                r['question_name'],
                np.array([r['STDNT_SEX_CD'],r['STDNT_CTZN_STAT_CD'], r['international'], 
                  r['White'],r['Asian'],r['NonWhiteOrAsian'],
                 r['age'],r['exclClassCumGPA'],
                r['Freshman'], r['Junior'], r['Sophomore'], r['Senior'],
                r['termCreditsGPA'], r['termCreditsNoGPA'],
                r['athlete_1'], r['honorsPro'],
                r['LSA'], r['programBusiness'], r['programEngineering'], 
                r['programInformation'], r['programOther'],
                r['HSCalculusTaken'],  r['highSchoolGPA'], 
                r['majorsCount'], r['minorsCount'],
                r['PREV_TERM_CUM_GPA'], 
                r['parentsGraduateEdu'], r['minorityGroup'],
                r['available_flashcards'],
                r['days_offset'],
                r['prev_time_elapsed'],
                r['time_lag']
              ]).transpose()
                ))

In [21]:
gc.collect()

20

In [22]:
# remove students who don't have make any interactions with the tool
toRemove = []
for index in grouped_data.index:
  if len(grouped_data[index][0]) <= 10:
    toRemove.append(index)
grouped_data = grouped_data.drop(index=toRemove)

In [23]:
dataframe.groupby('user_id')['start_practice'].agg('count').describe()

count    1110.000000
mean      550.263063
std       201.281353
min         2.000000
25%       438.250000
50%       520.500000
75%       637.750000
max      1500.000000
Name: start_practice, dtype: float64

In [24]:
toRemove

['FA 2018:252', 'WN 2018:295']

In [26]:
#SETTINGS -> can be modified at any time
MAXLENGTH = 50
EMBEDDING_DIM = 128
DENSE_NEURON = 16
GRU_NEURON = 32

In [27]:
#FEATURES_SIZE = 38
CHAPTER_SIZE = 38
SUB_CHAPTER_SIZE = 223
QUESTION_SIZE = 1069

In [28]:
dataframe['sub_chapter_label'].unique()

array([ 96,  88,  86,  28,  12,  82, 125,  32,  69,  33,  21, 115, 120,
       119,  91,  73,  57,  77, 117,  40,  74,  41,  68,  92,  39,  47,
       104,  53,  95,  49,  16,  50,  52,  55,   7,  15,  51, 103, 102,
       100,  94, 108,  11,   6, 118,  18, 215,  46, 216, 141,  27,  90,
       217, 105,  54, 218, 123,  17,  67,  63,  13, 101, 124, 219,  20,
       122,  58,   3,  59,  64, 106,  93,  65,  66,   9,  44,  30, 139,
        26,  25,   8, 138,   1,  14,   5,  85,  35,  34,  79, 220, 121,
        36,  31,  24, 112, 109, 110,  70,  45,  10,  56,  60, 145,  61,
       129, 144,  72,  71,  89,  87,  97,  62,   4,  38,  19, 150, 131,
       130, 132, 149, 128, 137,  75,  99, 143, 107, 147, 151, 152, 153,
       159, 157, 158, 156, 160, 161, 163, 164, 165, 166, 167, 168, 170,
       171, 169, 172, 174, 173, 175, 177, 176, 180, 178, 181, 182, 187,
       189, 190, 191, 195, 193, 192, 197, 198, 199, 202, 201, 200, 204,
       203, 205, 207, 206, 208, 209, 210, 162, 183, 185, 184, 18

In [29]:
#create dataset class
#to prepare it for train, valid, and test sets
from torch.utils.data import Dataset, DataLoader
class SPACE_DATASET(Dataset):
    def __init__(self, data, maxlength = 50):
        super(SPACE_DATASET, self).__init__()
        self.maxlength = maxlength
        self.data = data
        self.users = list()
        for user in data.index:
            self.users.append(user)

    def __len__(self):
        return len(self.users)
    
    def __getitem__(self, ix):
        user = self.users[ix]
        user = user
        target, term, ch_label, sub_ch_label, ques_name, features = self.data[user]
        
        #0s should be used as padding values
        ori_target = target.values 
        term = term.values
        ch_label = ch_label.values + 1
        sub_ch_label = sub_ch_label.values +1
        ques_name = ques_name.values + 1
        
        n = len(ch_label)

        # one hot for term
        term_encode = [0]*7
        term_encode[term[0]] = 1
        shifted_target= []

        
        # get  user interaction informations in the previous MAXLEN interactions
        if n > self.maxlength:
          ch_label = ch_label[-self.maxlength:]
          sub_ch_label = sub_ch_label[-self.maxlength:]
          ques_name = ques_name[-self.maxlength:]
          features = features[-self.maxlength:]
          target = ori_target[-self.maxlength:]
          shifted_target = ori_target[ (-self.maxlength - 1) :-1]
        else:
          ch_label = [0]*(self.maxlength - n)+list(ch_label[:])
          sub_ch_label = [0]*(self.maxlength - n)+list(sub_ch_label[:])
          ques_name = [0]*(self.maxlength - n)+list(ques_name[:])
          features = [[0]*len(features[0])]*(self.maxlength  - n)+list(features[:])
          target = [-1]*(self.maxlength - n) + list(ori_target[:])
          shifted_target = [-1]*(self.maxlength + 1 - n) + list(ori_target[:-1])

        new_features = []
        count = 0
        for f in features:
          temp = list(f)
          temp.extend(term_encode)
          # temp.append(shifted_target[count]) #uncomment this line for include previous response feature
          new_features.append(temp)
          count += 1
        features = new_features
        return ch_label,sub_ch_label,ques_name,target

In [30]:
SUB_CHAPTER_SIZE

223

In [31]:
# 5 fold cross validation
#HERE WE ONLY USE chapter
#sub_chapter and question
#without additional features
import torch
X = np.array(grouped_data.keys())
kfold = KFold(n_splits=5, shuffle=True)
train_losses = list()
train_aucs = list()
val_losses = list()
val_aucs = list()
train_eval = list()
test_eval = list()
for train, test in kfold.split(X):
    users_train, users_test =  X[train], X[test]
    n = len(users_test)//2
    users_test, users_val = users_test[:n], users_test[n: ]
    train_data_space = SPACE_DATASET(grouped_data[users_train], MAXLENGTH)
    val_data_space = SPACE_DATASET(grouped_data[users_val], MAXLENGTH)
    test_data_space = SPACE_DATASET(grouped_data[users_test], MAXLENGTH)
    #construct training input
    train_chapter=[]
    train_sub_chapter=[]
    train_question = []
    train_features=[]
    train_labels=[]
    for i in range(len(users_train)):
        user = train_data_space.__getitem__(i)
        train_chapter.append(user[0])
        train_sub_chapter.append(user[1]) 
        train_question.append(user[2])
        #train_features.append(user[3])
        train_labels.append(user[3])
    train_chapter = np.array(train_chapter)
    train_sub_chapter = np.array(train_sub_chapter)
    train_question = np.array(train_question)
    #train_features = np.array(train_features)
    train_labels= np.array(train_labels)[..., np.newaxis]

    #construct validation input
    val_chapter=[]
    val_sub_chapter=[]
    val_question = []
    #val_features=[]
    val_labels=[]
    for i in range(len(users_val)):
        user = val_data_space.__getitem__(i)
        val_chapter.append(user[0])
        val_sub_chapter.append(user[1]) 
        val_question.append(user[2])
        #val_features.append(user[3])
        val_labels.append(user[3])
    val_chapter = np.array(val_chapter)
    val_sub_chapter = np.array(val_sub_chapter)
    #val_features = np.array(val_features)
    val_question = np.array(val_question)
    val_labels= np.array(val_labels)[..., np.newaxis]

    # construct test input
    test_chapter=[]
    test_sub_chapter=[]
    #test_features=[]
    test_question=[]
    test_labels=[]
    for i in range(len(users_test)):
        user = test_data_space.__getitem__(i)
        test_chapter.append(user[0])
        test_sub_chapter.append(user[1]) 
        test_question.append(user[2])
        #test_features.append(user[3])
        test_labels.append(user[3])
    test_chapter = np.array(test_chapter)
    test_sub_chapter = np.array(test_sub_chapter)
    #test_features = np.array(test_features)
    test_question = np.array(test_question)
    test_labels= np.array(test_labels)[..., np.newaxis]

    # define loss function and evaluation metrics
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    acc = tf.keras.metrics.Accuracy()
    auc = tf.keras.metrics.AUC()

    def masked_bce(y_true, y_pred):
      flat_pred = y_pred
      flat_ground_truth = y_true
      label_mask = tf.math.not_equal(flat_ground_truth, -1)
      return bce(flat_ground_truth, flat_pred, sample_weight=label_mask)

    def masked_acc(y_true, y_pred):
      flat_pred = y_pred
      flat_ground_truth = y_true
      flat_pred = (flat_pred >= 0.5)
      label_mask = tf.math.not_equal(flat_ground_truth, -1)
      return acc(flat_ground_truth, flat_pred, sample_weight=label_mask)

    def masked_auc(y_true, y_pred):
      flat_pred = y_pred
      flat_ground_truth = y_true
      label_mask = tf.math.not_equal(flat_ground_truth, -1)
      return auc(flat_ground_truth, flat_pred, sample_weight=label_mask)

    # input layer
    input_chap = tf.keras.Input(shape=(MAXLENGTH))
    input_sub_chap = tf.keras.Input(shape=(MAXLENGTH))
    input_ques =  tf.keras.Input(shape=(MAXLENGTH))
    #input_features = tf.keras.Input(shape=(MAXLENGTH, FEATURES_SIZE))

    # embedding layer for categorical features
    embedding_chap = Embedding(input_dim = CHAPTER_SIZE, output_dim = EMBEDDING_DIM)(input_chap)
    embedding_sub_chap = Embedding(input_dim = SUB_CHAPTER_SIZE, output_dim = EMBEDDING_DIM)(input_sub_chap) 
    embedding_ques = Embedding(input_dim = QUESTION_SIZE, output_dim = EMBEDDING_DIM)(input_ques)       

    # dense layer for numeric features
    #dense_features = Dense(EMBEDDING_DIM,input_shape = (None, MAXLENGTH))(input_features)

    gru_output = tf.concat([embedding_chap, embedding_sub_chap, embedding_ques], axis = 2)

    pred = Dense(1, input_shape = (None, 3*EMBEDDING_DIM), activation='sigmoid')(gru_output)

    model = tf.keras.Model(
        inputs=[input_chap, input_sub_chap,input_ques],
        outputs=pred,
        name='logistic_regression'
    )

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    opt_adam = Adam(learning_rate = 0.005)
    model.compile(
        optimizer=opt_adam,
        loss= masked_bce,
        metrics = [masked_acc, masked_auc]
    )

    history = model.fit(
      [train_chapter, train_sub_chapter, train_question],
      train_labels,
      batch_size = 64,
      epochs = 100,
      validation_data=([val_chapter, val_sub_chapter, val_question], val_labels),
      callbacks=[callback]
    )
    val_losses.append(list(history.history['val_loss']))
    train_losses.append(list(history.history['loss']))
    val_aucs.append(list(history.history['val_masked_auc']))
    train_aucs.append(list(history.history['masked_auc']))
    train_score = model.evaluate([train_chapter, train_sub_chapter, train_question], train_labels)
    train_eval.append(train_score)
    test_score = model.evaluate([test_chapter, test_sub_chapter, test_question], test_labels)
    test_eval.append(test_score)
    print("Test: ", test_score)
    def reset_weights(model):
      for layer in model.layers: 
        if isinstance(layer, tf.keras.Model):
          reset_weights(layer)
          continue
        for k, initializer in layer.__dict__.items():
          if "initializer" not in k:
            continue
          # find the corresponding variable
          var = getattr(layer, k.replace("_initializer", ""))
          var.assign(initializer(var.shape, var.dtype))
    reset_weights(model)

Epoch 1/100
14/14 [==============================] - 3s 104ms/step - loss: 0.6334 - masked_acc: 0.6172 - masked_auc: 0.6216 - val_loss: 0.5587 - val_masked_acc: 0.7011 - val_masked_auc: 0.6968
Epoch 2/100
14/14 [==============================] - 0s 13ms/step - loss: 0.5186 - masked_acc: 0.7101 - masked_auc: 0.7126 - val_loss: 0.5324 - val_masked_acc: 0.7215 - val_masked_auc: 0.7381
Epoch 3/100
14/14 [==============================] - 0s 13ms/step - loss: 0.5109 - masked_acc: 0.7236 - masked_auc: 0.7450 - val_loss: 0.5297 - val_masked_acc: 0.7288 - val_masked_auc: 0.7555
Epoch 4/100
14/14 [==============================] - 0s 13ms/step - loss: 0.5005 - masked_acc: 0.7302 - masked_auc: 0.7587 - val_loss: 0.5358 - val_masked_acc: 0.7329 - val_masked_auc: 0.7645
Epoch 5/100
14/14 [==============================] - 0s 13ms/step - loss: 0.5049 - masked_acc: 0.7334 - masked_auc: 0.7664 - val_loss: 0.5331 - val_masked_acc: 0.7354 - val_masked_auc: 0.7701
Epoch 6/100
14/14 [====================

Epoch 9/100
14/14 [==============================] - 0s 14ms/step - loss: 0.4957 - masked_acc: 0.7429 - masked_auc: 0.7780 - val_loss: 0.5262 - val_masked_acc: 0.7433 - val_masked_auc: 0.7791
Epoch 10/100
14/14 [==============================] - 0s 14ms/step - loss: 0.4990 - masked_acc: 0.7434 - masked_auc: 0.7795 - val_loss: 0.5262 - val_masked_acc: 0.7438 - val_masked_auc: 0.7803
Epoch 11/100
14/14 [==============================] - 0s 14ms/step - loss: 0.4932 - masked_acc: 0.7440 - masked_auc: 0.7808 - val_loss: 0.5256 - val_masked_acc: 0.7442 - val_masked_auc: 0.7814
Epoch 12/100
4/4 [==============================] - 1s 5ms/step - loss: 0.5193 - masked_acc: 0.7448 - masked_auc: 0.7834
Test:  [0.5192693471908569, 0.7447560429573059, 0.7833977937698364]
Epoch 1/100
14/14 [==============================] - 2s 52ms/step - loss: 0.6350 - masked_acc: 0.5966 - masked_auc: 0.5974 - val_loss: 0.5672 - val_masked_acc: 0.6977 - val_masked_auc: 0.6967
Epoch 2/100
14/14 [======================

In [32]:
t_eval = np.array(test_eval)
print("test avg loss: ", np.mean(t_eval[:, 0]), "+/-" ,np.std(t_eval[:, 0]))
print("test avg acc: ", np.mean(t_eval[:, 1]),  "+/-" ,np.std(t_eval[:, 1]))
print("test avg auc: ", np.mean(t_eval[:, 2]), "+/-" ,np.std(t_eval[:, 2]))

test avg loss:  0.5061074256896972 +/- 0.009424129656835383
test avg acc:  0.7435900926589966 +/- 0.0017603831939310964
test avg auc:  0.7853693842887879 +/- 0.0016954044656041815


In [33]:
t_eval = np.array(train_eval)
print("train avg loss: ", np.mean(t_eval[:, 0]), "+/-" ,np.std(t_eval[:, 0]))
print("train avg acc: ", np.mean(t_eval[:, 1]),  "+/-" ,np.std(t_eval[:, 1]))
print("train avg auc: ", np.mean(t_eval[:, 2]), "+/-" ,np.std(t_eval[:, 2]))

train avg loss:  0.4976381480693817 +/- 0.0027479999967223853
train avg acc:  0.7437751889228821 +/- 0.0018452511957679563
train avg auc:  0.7853979110717774 +/- 0.0017344484964463861


In [34]:
model.save("logistic_maxlength_50.h5")